In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

In [15]:
from core.datasets.dataset_loading_utils import load_dataset_bert
from core.datasets.motion_bert_dataset import BERTMotionDataset, DATALoader
from core.models.BERT import BERT, BERTParams
from core.optimizer import get_optimizer
from torch.utils.data import DataLoader
from configs.config import get_cfg_defaults

path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/bert_12_768/bert_12_768.yaml"
cfg = get_cfg_defaults()
print("loading config from:", path)
cfg.merge_from_file(path)
cfg.freeze()

loading config from: /srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/bert_12_768/bert_12_768.yaml


In [21]:
ds = BERTMotionDataset("t2m" , "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionIndices" , 0 , 128 , mask_prob = 0.3 )

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23384/23384 [00:22<00:00, 1027.84it/s]

Total number of motions 23384


In [22]:
from core.models.text_encoders import T5
t5 = T5(128)


In [33]:
dl = DATALoader(ds , 4 , t5)

In [34]:
for batch in dl:
    break

In [37]:
batch["mask"].dtype

torch.bool

In [29]:
params = BERTParams()
bert = BERT(params).cuda()

In [42]:
mlm_loss_fnc = torch.nn.CrossEntropyLoss(ignore_index=bert.pad_index)

In [39]:
mask_lm_output , out = bert.forward(
                    batch["bert_input"].cuda(), batch["mask"].cuda()
                )

In [40]:
mask_lm_output.shape

torch.Size([4, 127, 1027])

In [41]:
out.shape

torch.Size([4, 768])

In [55]:
context_embed = torch.max(batch["context_embed"] , 1).values

In [48]:
mask_loss = mlm_loss_fnc(mask_lm_output.transpose(1, 2), batch["bert_label"][:,1:].cuda())
mask_loss

tensor(6.9230, device='cuda:0', grad_fn=<NllLoss2DBackward0>)